# Simple HTTP client (認証あり) を使ったファイル取得

- ここでは Simple HTTP fileserver の API から python にてファイルを取得する方法を示します

## ファイル取得の流れ (Private Repository)

1. Simple HTTP fileserver にブラウザでログインし "Token" > "Your Token" を取得してください
2. "Your Token" に示されている文字列をコピーし ```SimpleClient``` の引数に設定します(必要なのは初回だけです)
3. またJWTトークンを保管しておく場所を決めておきます (このトークンは認証許可を示す大事な token ですので、外部に漏れないように管理をお願いいたします)
4. JWTトークンは OneTime トークンなのでプログラムごとに別の保存場所を用意してください

## import

In [29]:
%load_ext autoreload
%autoreload 2
from simpleclient import SimpleClient
import io
from PIL import Image
import matplotlib.pyplot as plt
import time
import random

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 各種設定値と SimpleClient の初期化

In [30]:
SERVER_URL = "http://FQDN"
YOUR_TOKEN = "YOUR_TOKEN"
JSON_PATH = "auth.json"
MAX_SHOW_IMAGE_COUNT = 100

client = SimpleClient(url=SERVER_URL, jsonPath=JSON_PATH)

## 認証の実施

- "Your token" を使った初回認証を実施します
- 認証に成功すると ```JsonPath``` で指定した場所にサーバーへアクセスするための JWTトークン が保存されます
- 認証に失敗する場合は Exception が発生します
- 2回目以降は ```JsonPath``` に保存された token を自動的に読み込みます
- JWTトークン認証を解除したい場合は ```JsonPath``` に保存されたファイルを削除します

In [31]:
try:
    client.Authorize(YOUR_TOKEN)
    print("認証成功")
except:
    print("認証失敗")
    exit(0)

認証成功


## ファイルリクエストの実行とパフォーマンス

- 必要なファイルの URL を調べファイル取得リクエストを行います
- URL がディレクトリの場合はディレクトリに含まれるファイル一覧が JSON 形式でレスポンスされます
- URL がファイルの場合 Byte 列でレスポンスされます
- Byte列(画像) は PIL Image によって画像化できます

In [32]:
def RequestFile(requestUrl):
    execList = 0
    execGet = 0

    t0 = time.perf_counter()
    try:
        response = client.Request(requestUrl)
        t1 = time.perf_counter()
        execList += t1 - t0
    except Exception as e:
        print(e)
        exit(0)

    if "list" in response:
        count = 0
        files = list(filter(lambda x: not x["isDir"], response["list"]))
        if len(files) > MAX_SHOW_IMAGE_COUNT:
            files = random.sample(files, MAX_SHOW_IMAGE_COUNT)
        for file in files:
            if not file["isDir"]:
                t2 = time.perf_counter()
                inputImage = Image.open(client.Request(requestUrl + file["path"]))
                t3 = time.perf_counter()
                execGet += (t3 - t2)
                #plt.imshow(inputImage)
                #plt.show()

        print("[listall] %d, [get file count] %d, [exec time] get filelist: %f s, get data per image %f s" % (len(response["list"]), len(files), execList, execGet / len(files)))


### 認証不要の場合

In [33]:
requestUrl = "http://michibiki.mohri.info:4888/share/public/val2017/"
RequestFile(requestUrl)

[listall] 5000, [get file count] 100, [exec time] get filelist: 0.058908 s, get data per image 0.009261 s


### 認証が必要な場合

In [34]:
requestUrl = "http://michibiki.mohri.info:4888/share/private/private002/val2017/"
RequestFile(requestUrl)

[listall] 5000, [get file count] 100, [exec time] get filelist: 0.071262 s, get data per image 0.021098 s


- 結論として認証が必要な場合でもパフォーマンスには大きな影響がないことが分かる